[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/document_augmentation.ipynb)

# 通过问题生成增强文档检索的文档增强

## 概述

此实现演示了一种文本增强技术，该技术利用额外的问题生成来改进向量数据库中的文档检索。通过生成并合并与每个文本片段相关的各种问题，系统增强了标准检索过程，从而增加了找到可用作生成式问答上下文的相关文档的可能性。

## 动机

通过用相关问题丰富文本片段，我们旨在显著提高识别文档中最相关部分的准确性，这些部分包含用户查询的答案。

## 先决条件

此方法利用 OpenAI 的语言模型和嵌入。您需要一个 OpenAI API 密钥才能使用此实现。请确保已安装所需的 Python 包：

```
pip install langchain openai faiss-cpu PyPDF2 pydantic
```

## 关键组件

1. **PDF 处理和文本分块**：处理 PDF 文档并将其划分为可管理的文本片段。
2. **问题增强**：使用 OpenAI 的语言模型在文档和片段级别生成相关问题。
3. **向量存储创建**：使用 OpenAI 的嵌入模型计算文档的嵌入，并创建一个 FAISS 向量存储。
4. **检索和答案生成**：使用 FAISS 找到最相关的文档，并根据提供的上下文生成答案。

## 方法详情

### 文档预处理

1. 使用 LangChain 的 PyPDFLoader 将 PDF 转换为字符串。
2. 将文本拆分为重叠的文本文档（text_document）以用于构建上下文，然后将每个文档拆分为重叠的文本片段（text_fragment）以用于检索和语义搜索。

### 文档增强

1. 使用 OpenAI 的语言模型在文档或文本片段级别生成问题。
2. 使用 QUESTIONS_PER_DOCUMENT 常量配置要生成的问题数量。

### 向量存储创建

1. 使用 OpenAIEmbeddings 类计算文档嵌入。
2. 从这些嵌入创建一个 FAISS 向量存储。

### 检索和生成

1. 根据给定的查询从 FAISS 存储中检索最相关的文档。
2. 使用检索到的文档作为上下文，使用 OpenAI 的语言模型生成答案。

## 此方法的优点

1. **增强的检索过程**：增加了为给定查询找到最相关的 FAISS 文档的概率。
2. **灵活的上下文调整**：允许轻松调整文本文档和片段的上下文窗口大小。
3. **高质量的语言理解**：利用 OpenAI 强大的语言模型进行问题生成和答案生成。

## 实现细节

- `OpenAIEmbeddingsWrapper` 类为嵌入生成提供了一致的接口。
- `generate_questions` 函数使用 OpenAI 的聊天模型从文本中创建相关问题。
- `process_documents` 函数处理文档拆分、问题生成和向量存储创建的核心逻辑。
- 主执行演示了加载 PDF、处理其内容并执行示例查询。

## 结论

该技术提供了一种提高基于向量的文档搜索系统中信息检索质量的方法。通过生成类似于用户查询的其他问题并利用 OpenAI 的高级语言模型，它可能在后续任务（如问答）中带来更好的理解和更准确的响应。

## 关于 API 使用的说明

请注意，此实现使用 OpenAI 的 API，可能会根据使用情况产生费用。请确保监控您的 API 使用情况并在您的 OpenAI 帐户设置中设置适当的限制。

# 包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要包。


In [ ]:
# 安装所需的包
!pip install faiss-cpu langchain langchain-openai python-dotenv

In [ ]:
# 克隆存储库以访问辅助函数和评估模块
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# 如果您需要使用最新数据运行
# !cp -r RAG_TECHNIQUES/data .

In [3]:
import sys
import os
import re
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from enum import Enum
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from typing import Any, Dict, List, Tuple

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')


# 为 Colab 兼容性替换了原始路径附加

from helper_functions import *


class QuestionGeneration(Enum):
    """
    用于指定文档处理问题生成级别的枚举类。

    属性：
        DOCUMENT_LEVEL (int): 表示在整个文档级别生成问题。
        FRAGMENT_LEVEL (int): 表示在单个文本片段级别生成问题。
    """
    DOCUMENT_LEVEL = 1
    FRAGMENT_LEVEL = 2

#取决于模型，对于 Mitral 7B，最大可以是 8000，对于 Llama 3.1 8B，可以是 128k
DOCUMENT_MAX_TOKENS = 4000
DOCUMENT_OVERLAP_TOKENS = 100

#在较短文本上计算嵌入和文本相似度
FRAGMENT_MAX_TOKENS = 128
FRAGMENT_OVERLAP_TOKENS = 16

#在文档或片段级别生成的问题
QUESTION_GENERATION = QuestionGeneration.DOCUMENT_LEVEL
#将为特定文档或片段生成多少个问题
QUESTIONS_PER_DOCUMENT = 40

### 定义此管道使用的类和函数

In [10]:
class QuestionList(BaseModel):
    question_list: List[str] = Field(..., title="为文档或片段生成的问题列表")


class OpenAIEmbeddingsWrapper(OpenAIEmbeddings):
    """
    OpenAI 嵌入的包装类，提供与原始 OllamaEmbeddings 类似的接口。
    """
    
    def __call__(self, query: str) -> List[float]:
        """
        允许将实例用作可调用对象来为查询生成嵌入。

        参数：
            query (str): 要嵌入的查询字符串。

        返回：
            List[float]: 查询的嵌入，为浮点数列表。
        """
        return self.embed_query(query)

def clean_and_filter_questions(questions: List[str]) -> List[str]:
    """
    清理和过滤问题列表。

    参数：
        questions (List[str]): 要清理和过滤的问题列表。

    返回：
        List[str]: 以问号结尾的已清理和过滤的问题列表。
    """
    cleaned_questions = []
    for question in questions:
        cleaned_question = re.sub(r'^\d+\.\s*', '', question.strip())
        if cleaned_question.endswith('?'):
            cleaned_questions.append(cleaned_question)
    return cleaned_questions

def generate_questions(text: str) -> List[str]:
    """
    使用 OpenAI 根据提供的文本生成问题列表。

    参数：
        text (str): 用于生成问题的上下文数据。

    返回：
        List[str]: 唯一、已过滤的问题列表。
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        input_variables=["context", "num_questions"],
        template="Using the context data: {context}\n\nGenerate a list of at least {num_questions} "
                 "possible questions that can be asked about this context. Ensure the questions are "
                 "directly answerable within the context and do not include any answers or headers. "
                 "Separate the questions with a new line character."
    )
    chain = prompt | llm.with_structured_output(QuestionList)
    input_data = {"context": text, "num_questions": QUESTIONS_PER_DOCUMENT}
    result = chain.invoke(input_data)
    
    # 从 QuestionList 对象中提取问题列表
    questions = result.question_list
    
    filtered_questions = clean_and_filter_questions(questions)
    return list(set(filtered_questions))

def generate_answer(content: str, question: str) -> str:
    """
    使用 OpenAI 根据提供的上下文为给定问题生成答案。

    参数：
        content (str): 用于生成答案的上下文数据。
        question (str): 为其生成答案的问题。

    返回：
        str: 基于所提供上下文的对问题的精确回答。
    """
    llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="Using the context data: {context}\n\nProvide a brief and precise answer to the question: {question}"
    )
    chain =  prompt | llm
    input_data = {"context": content, "question": question}
    return chain.invoke(input_data)

def split_document(document: str, chunk_size: int, chunk_overlap: int) -> List[str]:
    """
    将文档拆分为更小的文本块。

    参数：
        document (str): 要拆分的文档的文本。
        chunk_size (int): 每个块的大小（以令牌数为单位）。
        chunk_overlap (int): 连续块之间的重叠令牌数。

    返回：
        List[str]: 文本块列表，其中每个块是文档内容的字符串。
    """
    tokens = re.findall(r'\b\w+\b', document)
    chunks = []
    for i in range(0, len(tokens), chunk_size - chunk_overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        chunks.append(chunk_tokens)
        if i + chunk_size >= len(tokens):
            break
    return [" ".join(chunk) for chunk in chunks]

def print_document(comment: str, document: Any) -> None:
    """
    打印注释，后跟文档内容。

    参数：
        comment (str): 在文档详细信息之前打印的注释或描述。
        document (Any): 要打印其内容的文档。

    返回：
        None
    """
    print(f'{comment} (type: {document.metadata["type"]}, index: {document.metadata["index"]}): {document.page_content}')

### 示例用法


In [ ]:
# 初始化 OpenAIEmbeddings
embeddings = OpenAIEmbeddingsWrapper()

# 示例文档
example_text = "This is an example document. It contains information about various topics."

# 生成问题
questions = generate_questions(example_text)
print("Generated Questions:")
for q in questions:
    print(f"- {q}")

# 生成答案
sample_question = questions[0] if questions else "What is this document about?"
answer = generate_answer(example_text, sample_question)
print(f"\nQuestion: {sample_question}")
print(f"Answer: {answer}")

# 拆分文档
chunks = split_document(example_text, chunk_size=10, chunk_overlap=2)
print("\nDocument Chunks:")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}: {chunk}")

# 使用 OpenAIEmbeddings 的示例
doc_embedding = embeddings.embed_documents([example_text])
query_embedding = embeddings.embed_query("What is the main topic?")
print("\nDocument Embedding (first 5 elements):", doc_embedding[0][:5])
print("Query Embedding (first 5 elements):", query_embedding[:5])

### 主管道

In [12]:
def process_documents(content: str, embedding_model: OpenAIEmbeddings):
    """
    处理文档内容，将其拆分为片段，生成问题，
    创建 FAISS 向量存储，并返回一个检索器。

    参数：
        content (str): 要处理的文档内容。
        embedding_model (OpenAIEmbeddings): 用于向量化的嵌入模型。

    返回：
        VectorStoreRetriever: 一个用于检索最相关 FAISS 文档的检索器。
    """
    # 将整个文本内容拆分为文本文档
    text_documents = split_document(content, DOCUMENT_MAX_TOKENS, DOCUMENT_OVERLAP_TOKENS)
    print(f'Text content split into: {len(text_documents)} documents')

    documents = []
    counter = 0
    for i, text_document in enumerate(text_documents):
        text_fragments = split_document(text_document, FRAGMENT_MAX_TOKENS, FRAGMENT_OVERLAP_TOKENS)
        print(f'Text document {i} - split into: {len(text_fragments)} fragments')
        
        for j, text_fragment in enumerate(text_fragments):
            documents.append(Document(
                page_content=text_fragment,
                metadata={"type": "ORIGINAL", "index": counter, "text": text_document}
            ))
            counter += 1
            
            if QUESTION_GENERATION == QuestionGeneration.FRAGMENT_LEVEL:
                questions = generate_questions(text_fragment)
                documents.extend([
                    Document(page_content=question, metadata={"type": "AUGMENTED", "index": counter + idx, "text": text_document})
                    for idx, question in enumerate(questions)
                ])
                counter += len(questions)
                print(f'Text document {i} Text fragment {j} - generated: {len(questions)} questions')
        
        if QUESTION_GENERATION == QuestionGeneration.DOCUMENT_LEVEL:
            questions = generate_questions(text_document)
            documents.extend([
                Document(page_content=question, metadata={"type": "AUGMENTED", "index": counter + idx, "text": text_document})
                for idx, question in enumerate(questions)
            ])
            counter += len(questions)
            print(f'Text document {i} - generated: {len(questions)} questions')

    for document in documents:
        print_document("Dataset", document)

    print(f'Creating store, calculating embeddings for {len(documents)} FAISS documents')
    vectorstore = FAISS.from_documents(documents, embedding_model)

    print("Creating retriever returning the most relevant FAISS document")
    return vectorstore.as_retriever(search_kwargs={"k": 1})

### 示例

In [ ]:
# 下载所需的数据文件
import os
os.makedirs('data', exist_ok=True)

# 下载本笔记本中使用的 PDF 文档
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [ ]:

# 将示例 PDF 文档加载到字符串变量中
path = "data/Understanding_Climate_Change.pdf"
content = read_pdf_to_string(path)

# 实例化将由 FAISS 使用的 OpenAI Embeddings 类
embedding_model = OpenAIEmbeddings()

# 处理文档并创建检索器
document_query_retriever = process_documents(content, embedding_model)

# 检索器使用示例
query = "What is climate change?"
retrieved_docs = document_query_retriever.get_relevant_documents(query)
print(f"\nQuery: {query}")
print(f"Retrieved document: {retrieved_docs[0].page_content}")

### 在存储中查找最相关的 FAISS 文档。在大多数情况下，这将是一个增强的问题，而不是原始文本文档。

In [ ]:
query = "How do freshwater ecosystems change due to alterations in climatic factors?"
print (f'Question:{os.linesep}{query}{os.linesep}')
retrieved_documents = document_query_retriever.invoke(query)

for doc in retrieved_documents:
    print_document("Relevant fragment retrieved", doc)

### 查找父文本文档并将其用作生成模型生成问题答案的上下文。

In [ ]:
context = doc.metadata['text']
print (f'{os.linesep}Context:{os.linesep}{context}')
answer = generate_answer(context, query)
print(f'{os.linesep}Answer:{os.linesep}{answer}')